## NCCN Guideline Chat Query Interface

In [1]:
#!pip install PyPDF
#!pip install pdfplumber
#%pip install langchain
#%pip install langchain-community
#%pip install langchain-openai
#%pip install langchain-chroma
#%pip install fastapi
#%pip install pypdf
#%pip install gradio

### Import the necessary libraries

In [2]:
import os
import glob
from dotenv import load_dotenv
import gradio as gr
import langchain
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import matplotlib.pyplot as plt
#from sklearn.manifold import TSNE
import numpy as np
#import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings
#from langchain_community.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader, PyPDFLoader, PDFPlumberLoader, UnstructuredPDFLoader, PDFMinerLoader
from pypdf.errors import PdfReadError

### Load the API keys for OpenAI and HuggingFace

In [3]:
load_dotenv(dotenv_path="../keys.env")
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['HF_API_KEY'] = os.getenv("HF_API_KEY")

### Define the OpenAI LLM that will be used.  Set the name of the database

In [4]:
# Use a low cost model from OpenAI will be sufficient

MODEL = "gpt-4o-mini"
db_name = "nccn_guidelines_db"

### Load in the PDF documents from the NCCN folder. A variety of PDF loaders will stand ready if there is a problem with one of them. 

In [5]:
# Read in documents

folder = "NCCN"  # Your folder containing PDFs

pdf_files = [f for f in os.listdir(folder) if f.endswith('.pdf')]

def add_metadata(doc, doc_type):
    doc.metadata["doc_type"] = doc_type
    return doc

# With thanks to CG and Jon R, students on the course, for this fix needed for some users 
text_loader_kwargs = {'encoding': 'utf-8'}
# If that doesn't work, some Windows users might need to uncomment the next line instead
# text_loader_kwargs={'autodetect_encoding': True}

def try_multiple_loaders(file_path):
    loaders = [
        (PyPDFLoader, "PyPDFLoader"),
        (PDFPlumberLoader, "PDFPlumberLoader"),
        (UnstructuredPDFLoader, "UnstructuredPDFLoader"),
        (PDFMinerLoader, "PDFMinerLoader")
    ]
    
    for LoaderClass, loader_name in loaders:
        try:
            loader = LoaderClass(file_path)
            docs = loader.load()
            print(f"Success with {loader_name} for {file_path}")
            return docs
        except Exception as e:
            print(f"{loader_name} failed: {str(e)}")
            continue
    
    return None  # If all loaders fail
    

documents = []
for pdf_file in pdf_files:
    full_path = os.path.join(folder, pdf_file)
    docs = try_multiple_loaders(full_path)
    if docs:
        doc_type = pdf_file  # or pdf_file.replace('.pdf', '') if you don't want the extension
        documents.extend([add_metadata(doc, doc_type) for doc in docs])
    else:
        print(f"All loaders failed for {pdf_file}")

print ("PDF ingestion completed.")

Success with PyPDFLoader for NCCN/cml2025.pdf
Success with PyPDFLoader for NCCN/anal2025.pdf
Success with PyPDFLoader for NCCN/biliary2025.pdf
Success with PyPDFLoader for NCCN/myeloma2025.pdf
Success with PyPDFLoader for NCCN/cervical2025.pdf
Success with PyPDFLoader for NCCN/uveal2025.pdf
Success with PyPDFLoader for NCCN/gist2025.pdf
Success with PyPDFLoader for NCCN/mpn2025.pdf
Success with PyPDFLoader for NCCN/cll_sll2025.pdf
Success with PyPDFLoader for NCCN/mds2025.pdf
Success with PyPDFLoader for NCCN/ovarian2025.pdf
Success with PyPDFLoader for NCCN/hcc2025.pdf
Success with PyPDFLoader for NCCN/neuroendocrine2025.pdf
Success with PyPDFLoader for NCCN/squamous2025.pdf
Success with PyPDFLoader for NCCN/colon2025.pdf
Success with PyPDFLoader for NCCN/thyroid2025.pdf
Success with PyPDFLoader for NCCN/small_bowel2024.pdf
Success with PyPDFLoader for NCCN/uterine2025.pdf
Success with PyPDFLoader for NCCN/kaposi2025.pdf
Success with PyPDFLoader for NCCN/sclc2025.pdf
Success with PyPD

### Here is where we define the chunk size.

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=60, separators=["\n\n", "\n", ".", ", "])
chunks = text_splitter.split_documents(documents)

### Looks at the "doc_type' of the metadata and identifies document types, such as PDF (or TXT). If PDF files were in the input stack, then one should see PDF files. This is quality control.

In [7]:
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")

Document types found: gastric2025.pdf, esophageal2025.pdf, nsclc2025.pdf, pancreatic2025.pdf, head-and-neck2025.pdf, colon2025.pdf, kaposi2025.pdf, thymic2025.pdf, occult2025.pdf, uveal2025.pdf, mds2025.pdf, testicular2025.pdf, NHL_T-cell2025.pdf, hcc2025.pdf, small_bowel2024.pdf, thyroid2025.pdf, hodgkins2025.pdf, ovarian2025.pdf, ampullary2025.pdf, anal2025.pdf, myeloma2025.pdf, bone2025.pdf, histiocytic_neoplasms2025.pdf, biliary2025.pdf, basal2025.pdf, cml2025.pdf, rectal2025.pdf, breast2025.pdf, mcc2025.pdf, mpn2025.pdf, gist2025.pdf, aml2025.pdf, uterine2025.pdf, cns2025.pdf, cutaneous_melanoma2025.pdf, cervical2025.pdf, neuroendocrine2025.pdf, sclc2025.pdf, all2024.pdf, gtn2025.pdf, cll_sll2025.pdf, sarcoma2025.pdf, vaginal2025.pdf, NHL_B-cell2025.pdf, vulvar2024.pdf, kidney2025.pdf, squamous2025.pdf, bladder2025.pdf, waldenstroms2025.pdf, prostate2025.pdf


### This code displays unique filenames instead of document types. 

In [8]:
unique_files = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Files processed: {', '.join(unique_files)}")
print(f"Total unique files: {len(unique_files)}")

Files processed: gastric2025.pdf, esophageal2025.pdf, nsclc2025.pdf, pancreatic2025.pdf, head-and-neck2025.pdf, colon2025.pdf, kaposi2025.pdf, thymic2025.pdf, occult2025.pdf, uveal2025.pdf, mds2025.pdf, testicular2025.pdf, NHL_T-cell2025.pdf, hcc2025.pdf, small_bowel2024.pdf, thyroid2025.pdf, hodgkins2025.pdf, ovarian2025.pdf, ampullary2025.pdf, anal2025.pdf, myeloma2025.pdf, bone2025.pdf, histiocytic_neoplasms2025.pdf, biliary2025.pdf, basal2025.pdf, cml2025.pdf, rectal2025.pdf, breast2025.pdf, mcc2025.pdf, mpn2025.pdf, gist2025.pdf, aml2025.pdf, uterine2025.pdf, cns2025.pdf, cutaneous_melanoma2025.pdf, cervical2025.pdf, neuroendocrine2025.pdf, sclc2025.pdf, all2024.pdf, gtn2025.pdf, cll_sll2025.pdf, sarcoma2025.pdf, vaginal2025.pdf, NHL_B-cell2025.pdf, vulvar2024.pdf, kidney2025.pdf, squamous2025.pdf, bladder2025.pdf, waldenstroms2025.pdf, prostate2025.pdf
Total unique files: 50


### A more comprehensive summary of the loading and chunking of the files. 

In [9]:
# First, let's see what files we started with
print(f"Total PDFs found: {len(pdf_files)}")
print(f"PDF files: {', '.join(pdf_files)}")

# After processing, let's see what made it into documents
doc_sources = set(doc.metadata['source'] for doc in documents)
print(f"\nSuccessfully processed documents: {len(doc_sources)}")
print(f"Sources: {', '.join(doc_sources)}")

# After chunking, let's see what we have
chunk_sources = set(chunk.metadata['source'] for chunk in chunks)
print(f"\nDocuments that were chunked: {len(chunk_sources)}")
print(f"Chunked sources: {', '.join(chunk_sources)}")

# Create sets of files
original_files = set(pdf_files)
processed_files = set(os.path.basename(source) for source in doc_sources)

# Find failed files
failed_files = original_files - processed_files
print("\nFiles that failed to process:")
if not failed_files:  # This checks if the set is empty
    print("No failed files. All were successfully processed.")
else:
    for file in failed_files:
        print(f"- {file}")

Total PDFs found: 50
PDF files: cml2025.pdf, anal2025.pdf, biliary2025.pdf, myeloma2025.pdf, cervical2025.pdf, uveal2025.pdf, gist2025.pdf, mpn2025.pdf, cll_sll2025.pdf, mds2025.pdf, ovarian2025.pdf, hcc2025.pdf, neuroendocrine2025.pdf, squamous2025.pdf, colon2025.pdf, thyroid2025.pdf, small_bowel2024.pdf, uterine2025.pdf, kaposi2025.pdf, sclc2025.pdf, bladder2025.pdf, bone2025.pdf, nsclc2025.pdf, vaginal2025.pdf, esophageal2025.pdf, NHL_B-cell2025.pdf, cutaneous_melanoma2025.pdf, sarcoma2025.pdf, kidney2025.pdf, all2024.pdf, head-and-neck2025.pdf, aml2025.pdf, vulvar2024.pdf, gtn2025.pdf, waldenstroms2025.pdf, rectal2025.pdf, mcc2025.pdf, prostate2025.pdf, testicular2025.pdf, thymic2025.pdf, basal2025.pdf, breast2025.pdf, hodgkins2025.pdf, NHL_T-cell2025.pdf, histiocytic_neoplasms2025.pdf, pancreatic2025.pdf, ampullary2025.pdf, gastric2025.pdf, occult2025.pdf, cns2025.pdf

Successfully processed documents: 50
Sources: NCCN/cml2025.pdf, NCCN/gtn2025.pdf, NCCN/bladder2025.pdf, NCCN/sclc

### Here each chunk of text is mapped into a vector that represents the meaning of the text. This is known as an embedding, and OpenAI's embedding method will be used. The vectors will be stored in a vector database.

In [10]:
# Put the chunks of data into a Vector Store that associates a Vector Embedding with each chunk
# Chroma is a popular open source Vector Database based on SQLLite

from tqdm import tqdm  # for progress bar

def create_vectorstore_in_batches(chunks, embedding, batch_size=100, db_name="db"):
    total_batches = (len(chunks) + batch_size - 1) // batch_size  # Calculate total number of batches
    
    # Initialize the vector store with the first batch
    first_batch = chunks[:batch_size]
    vectorstore = Chroma.from_documents(
        documents=first_batch,
        embedding=embedding,
        persist_directory=db_name,
        collection_metadata={"hnsw:space": "cosine"}
    )
    
    # Process the remaining chunks in batches
    with tqdm(total=total_batches-1, desc="Processing batches", ncols=150) as pbar:
        for i in range(batch_size, len(chunks), batch_size):
            batch = chunks[i:i + batch_size]
            vectorstore.add_documents(documents=batch)
            pbar.update(1)
    
    return vectorstore

# Use it like this:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vectorstore = create_vectorstore_in_batches(chunks, embeddings, batch_size=100, db_name=db_name)

Processing batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████| 584/584 [18:38<00:00,  1.91s/it]


### Chroma creates collections in the vector database the holds information, such as the embeddings (vectors), associated metadata, documents/text, and IDs for each entry.

In [11]:
collection = vectorstore._collection
count = collection.count()

sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"There are {count:,} vectors with {dimensions:,} dimensions in the vector store")

There are 116,994 vectors with 3,072 dimensions in the vector store


## Create the chat model using OpenAI or a local open-source model. Create the retriever and conversation chain. 

In [12]:
# create a new Chat with OpenAI
llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

# Alternative - if you'd like to use Ollama locally, uncomment this line instead
#llm = ChatOpenAI(temperature=0.7, model_name='llama3.3', base_url='http://localhost:11434/v1', api_key='ollama')

# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG
retriever = vectorstore.as_retriever(search_kwargs={"k":5})

# putting it together: set up the conversation chain with the GPT 3.5 LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

/var/folders/wz/jqsk5sjd6_95g21hssmwrx7r0000gn/T/ipykernel_29069/3145502156.py:8: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)


### Run a quick test with a simple query.

In [13]:
from IPython.display import Markdown
# Let's try a simple question

query = "If a breast cancer patient fails fulvestrant, what would be suitable options for next-line therapy?"
result = conversation_chain.invoke({"question": query})
display(Markdown(result["answer"]))

If a breast cancer patient fails fulvestrant, suitable options for next-line therapy may include:

1. A CDK 4/6 inhibitor (such as palbociclib, ribociclib, or abemaciclib) in combination with another hormonal therapy, if not previously used.
2. Everolimus in combination with either an aromatase inhibitor (AI), tamoxifen, or fulvestrant.
3. Monotherapy with another hormonal therapy, depending on the specific clinical scenario.

It's important to consider the patient's overall health, prior treatments, and specific cancer characteristics when deciding on the next-line therapy.

### Set up conversational memory.

In [14]:
# set up a new conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

retriever = vectorstore.as_retriever(search_kwargs={"k": 25})

# putting it together: set up the conversation chain with the GPT 4o-mini LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

### Create a chat function

In [15]:
def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]

### Use Gradio for a chat user interface.

In [16]:
view = gr.ChatInterface(chat, 
    type="messages",
    css="""textarea {
        font-size: 18px !important;
    }
    .message-wrap textarea {
        font-size: 18px !important;
    }
    .message-content {
        font-size: 18px !important;
    }
    .chatbot.prose {
        font-size: 18px !important;
    }
    .svelte-7ddecg {
        font-size 18px !important;
    }
    """).launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


### This gives a behind-the-scenes peek into the "thought process" of the LLM.

In [17]:
from langchain_core.callbacks import StdOutCallbackHandler

llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

retriever = vectorstore.as_retriever()

conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory, callbacks=[StdOutCallbackHandler()])

# Enter a simple sample query.
query = "What is stage is a T3,N1 lung cancer?"
result = conversation_chain.invoke({"question": query})
answer = result["answer"]
print("\nAnswer:", answer)



> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Stage I, centrale (T1abc–T2a, N0);  
Stage II (T1abc–T2ab, N1; T2b, N0); 
Stage IIB (T3, N0)f; stage IIIA (T3, N1)f
Stage IIB (T3 invasion, N0);
Stage IIIA (T4 invasion, N0–1; 
T3, N1; T4, N0–1)
Stage IIIA (T1–2, N2); stage IIIB (T3, N2) 
Separate pulmonary nodule(s) (stage IIB, IIIA, IV) 
Stage IIIA ipsilateral non-primary lobe (T4, N0–1); 
Stage IV (contralateral lung) 
Multiple lung cancers
Stage IIIB (T1–2, N3); stage IIIC (T3, N3) 
Stage IIIB (T4, N2); stage IIIC (T4, N3)

Stage I, centrale (T1abc–T2a, N0);  
Stage II (T1abc–T2ab, N1; T2b, N0); 
Stage IIB (T3, N0)f; stage IIIA (T3, N1)f
Stage IIB (T3 invasion, N0);
Stage IIIA (T4 invasion, N

In [18]:
# create a new Chat with OpenAI
llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG; k is how many chunks to use
retriever = vectorstore.as_retriever(search_kwargs={"k": 20})

# putting it together: set up the conversation chain with the GPT 3.5 LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [19]:
def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]

In [20]:
view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
